Import LAION Model

In [ ]:
import os
import torch
import torch.nn as nn
from PIL import Image
import open_clip
from os.path import expanduser
from urllib.request import urlretrieve
import pandas as pd

def get_aesthetic_model(clip_model="vit_l_14"):
    """Load the aesthetic model"""
    home = expanduser("~")
    cache_folder = os.path.join(home, ".cache", "emb_reader")
    path_to_model = os.path.join(cache_folder, f"sa_0_4_{clip_model}_linear.pth")
    if not os.path.exists(path_to_model):
        os.makedirs(cache_folder, exist_ok=True)
        url_model = (
            f"https://github.com/LAION-AI/aesthetic-predictor/blob/main/sa_0_4_{clip_model}_linear.pth?raw=true"
        )
        urlretrieve(url_model, path_to_model)
    if clip_model == "vit_l_14":
        m = nn.Linear(768, 1)
    elif clip_model == "vit_b_32":
        m = nn.Linear(512, 1)
    else:
        raise ValueError("Unsupported CLIP model type")
    s = torch.load(path_to_model)
    m.load_state_dict(s)
    m.eval()
    return m


# Load the aesthetic model
amodel = get_aesthetic_model(clip_model="vit_l_14")
amodel.eval()

# Load the CLIP model
model, _, preprocess = open_clip.create_model_and_transforms('ViT-L-14', pretrained='openai')
model.eval()

def process_images_in_directory(directory_path):
    results = []
    # Iterate through all files in the directory
    for filename in os.listdir(directory_path):
        if filename.lower().endswith((".jpg", ".jpeg", ".png", ".bmp")):  # Add or remove file extensions as needed
            image_path = os.path.join(directory_path, filename)

            # Open and preprocess the image  revise the code for my following instructions 
            image = preprocess(Image.open(image_path)).unsqueeze(0)

            # Process the image
            with torch.no_grad():
                image_features = model.encode_image(image)
                image_features /= image_features.norm(dim=-1, keepdim=True)
                prediction = amodel(image_features)

                score = prediction.item()
                results.append({"Filename": filename, "Aesthetic Score": score})

                print(f"Aesthetic prediction for {filename}:")
                print(score)
                print("-------------------------")

    # Create a DataFrame from the results
    df = pd.DataFrame(results)

    # Calculate the mean score
    mean_score = df["Aesthetic Score"].mean()
    print(f"Mean Aesthetic Score: {mean_score}")

    # Add the mean score to the DataFrame
    df = df._append({"Filename": "Mean Score", "Aesthetic Score": mean_score}, ignore_index=True)

    # Save to Excel
    foldername = os.path.basename(folder_path)
    excel_path = os.path.join(directory_path, f"{foldername}.xlsx")
    df.to_excel(excel_path, index=False)
    print(f"Results saved to {excel_path}")

C:\Users\ICtSP\AppData\Local\Temp\ipykernel_27436\1737881975.py:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  s = torch.load(path_to_model)


CLIP(
  (visual): VisionTransformer(
    (conv1): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
    (patch_dropout): Identity()
    (ln_pre): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): ModuleList(
        (0-23): 24 x ResidualAttentionBlock(
          (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=1024, out_features=1024, bias=True)
          )
          (ls_1): Identity()
          (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=1024, out_features=4096, bias=True)
            (gelu): QuickGELU()
            (c_proj): Linear(in_features=4096, out_features=1024, bias=True)
          )
          (ls_2): Identity()
        )
      )
    )
    (ln_post): LayerNorm((1024,), eps=1e-05, elementwise_affine=Tru

Set directory

In [ ]:
folder_path='d:/Frank/Fall work/Prompt test'
directory_path = 'd:/Frank/Fall work/Prompt test'
process_images_in_directory(directory_path)

Sort the excel in numerical order

In [52]:
import pandas as pd
import re

def natural_sort_key(s):
    # Split the string into a list of strings and numbers
    return [int(text) if text.isdigit() else text.lower() for text in re.split('(\d+)', s)]

# Loop through the range from 1 to 4
for i in range(1,2):
    # Use the loop variable to dynamically set the file paths
    file_path = f'd:/Frank/Fall work/generated_images/xl/only no greg/negative/{i}/{i}.xlsx'
    df = pd.read_excel(file_path)

    # Sort the DataFrame by the first column using the custom natural sort key
    df_sorted = df.sort_values(by=df.columns[0], key=lambda col: col.map(natural_sort_key))

    # Save the sorted DataFrame to a new Excel file
    new_file_path = f'd:/Frank/Fall work/generated_images/xl/only no greg/negative/{i}/sorted_{i}.xlsx'
    df_sorted.to_excel(new_file_path, index=False)

    print(f'Sorted Excel file has been saved as {new_file_path}')


Sorted Excel file has been saved as d:/Frank/Fall work/generated_images/xl/only no greg/negative/1/sorted_1.xlsx


Remove black

In [55]:
import pandas as pd
import numpy as np

# Load the Excel file
file_path = 'd:/Frank/Fall work/generated_images/LAION Score.xlsx'
df = pd.read_excel(file_path)

# Replace values that start with "3.357" up to a certain precision with NaN
df = df.applymap(lambda x: np.nan if isinstance(x, float) and str(x).startswith('3.357') else x)

# Save the modified DataFrame to a new Excel file
new_file_path = 'd:/Frank/Fall work/generated_images/modified_LAION_Score.xlsx'
df.to_excel(new_file_path, index=False)

print(f'Modified Excel file has been saved as {new_file_path}')


C:\Users\ICtSP\AppData\Local\Temp\ipykernel_26416\3673909037.py:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: np.nan if isinstance(x, float) and str(x).startswith('3.357') else x)


Modified Excel file has been saved as d:/Frank/Fall work/generated_images/modified_LAION_Score.xlsx
